In [1]:
import requests
import time
import csv
import pandas as pd
import json
import concurrent.futures

In [2]:
output_path = '/home/phuonn/Shopee_food/UDPTDLTM_DATA/data/menu_data.csv'

In [3]:
delivery_df = pd.read_csv('data/delivery_info_clean.csv')

In [4]:
id_list = delivery_df[['id', 'restaurant_id']]

In [7]:
def request_dish_detail(restaurant_id):
  params = {
        "id_type": 2,
        "request_id": restaurant_id
  }
  headers={
        "X-Foody-Access-Token": "",  
        "X-Foody-Api-Version": "1",
        "X-Foody-App-Type": "1004",
        "X-Foody-Client-Id": "",  
        "X-Foody-Client-Language": "vi",
        "X-Foody-Client-Type": "1",
        "X-Foody-Client-Version": "3.0.0"
  }
  url = "https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes"
  return requests.get(url, headers=headers, params=params)

In [8]:
def get_dish_details(restaurant_id):
    """Gửi yêu cầu API và lấy thông tin món ăn cho một restaurant_id."""
    retries = 3
    dishes_list = []

    for attempt in range(retries):
        try:
            response = request_dish_detail(restaurant_id)
            response.raise_for_status() 
            data = response.json()
            menu_infos = data.get('reply', {}).get('menu_infos', [])
    
            has_dishes = False 

            for menu in menu_infos:
                dishes = menu.get('dishes', [])
                if dishes: 
                    has_dishes = True

                for dish in dishes:
                    name = dish.get('name', '')
                    price = dish.get('price', {}).get('value', '')
                    if name and price:  
                        dishes_list.append({'Name': name, 'Price': price})

            if has_dishes:
                break
            else:
                print(f"No valid dishes for restaurant_id {restaurant_id}. Retrying...")
                time.sleep(2)

        except Exception as e:
            print(f"Lỗi khi lấy dữ liệu từ API cho restaurant_id {restaurant_id}: {e}")

        if attempt < retries - 1:
            time.sleep(1) 

    return {'restaurant_id': restaurant_id, 'menu': dishes_list}

In [9]:
def write_to_csv(output_csv, data):
    """Ghi dữ liệu vào file CSV với định dạng JSON dễ đọc."""
    records = []
    for item in data:
        restaurant_id = item['restaurant_id']
        menu = item['menu']
        
        # Chuyển đổi menu thành chuỗi JSON dễ đọc với các ký tự đặc biệt
        menu_str = json.dumps(menu, ensure_ascii=False)
        records.append({"restaurant_id": restaurant_id, "menu": menu_str})
    
    df = pd.DataFrame(records)
    
    # Ghi DataFrame vào file CSV với định dạng như yêu cầu
    df.to_csv(output_csv, mode='a', header=False, index=False, quoting=csv.QUOTE_NONNUMERIC)

In [10]:
def initialize_csv(output_csv=output_path):
    with open(output_csv, 'w') as f:
        f.write("restaurant_id,menu\n")

In [11]:
def get_dishes(id_list, output_csv=output_path):
    """Lấy dữ liệu món ăn cho danh sách restaurant_id."""
    initialize_csv(output_csv) 
    results = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        # Tạo danh sách các futures
        futures = {
            executor.submit(get_dish_details, int(row['restaurant_id'])): int(row['restaurant_id'])
            for index, row in id_list.iterrows()
        }

        # Chờ tất cả các futures hoàn thành
        for future in concurrent.futures.as_completed(futures):
            restaurant_id = futures[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as exc:
                print(f"Restaurant {restaurant_id} generated an exception: {exc}")

    write_to_csv(output_csv, results)  #

In [ ]:
get_dishes(id_list)

Lỗi khi lấy dữ liệu từ API cho restaurant_id 73155: 'NoneType' object has no attribute 'get'
Lỗi khi lấy dữ liệu từ API cho restaurant_id 103582: 'NoneType' object has no attribute 'get'
Lỗi khi lấy dữ liệu từ API cho restaurant_id 185899: 'NoneType' object has no attribute 'get'
Lỗi khi lấy dữ liệu từ API cho restaurant_id 88: 'NoneType' object has no attribute 'get'
Lỗi khi lấy dữ liệu từ API cho restaurant_id 3832: 'NoneType' object has no attribute 'get'
Lỗi khi lấy dữ liệu từ API cho restaurant_id 2682: 'NoneType' object has no attribute 'get'
Lỗi khi lấy dữ liệu từ API cho restaurant_id 6336: 'NoneType' object has no attribute 'get'
Lỗi khi lấy dữ liệu từ API cho restaurant_id 48683: 'NoneType' object has no attribute 'get'
Lỗi khi lấy dữ liệu từ API cho restaurant_id 5491: 'NoneType' object has no attribute 'get'
Lỗi khi lấy dữ liệu từ API cho restaurant_id 124860: 'NoneType' object has no attribute 'get'
Lỗi khi lấy dữ liệu từ API cho restaurant_id 5759: 'NoneType' object has n